<!--NAVIGATION-->
< [Introduction to NumPy](02.00-Introduction-to-NumPy.ipynb) | [Contents](Index.ipynb) | [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb) >

# 理解Python中的数据类型

本节将介绍在Python语言中数据数组是如何被存储和操作的，并对比NumPy所做的改进。

静态类型语言（如C或Java）需要对每一个变量的类型进行明确的定义，而动态类型的语言（例如Python）可以跳过这个特殊规定。Python好用在其**动态输入**。
举例：C语言中，你可能会按照如下方式指定一个特殊的操作：

```C
/* C code */
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```

而在Python中，同等的操作可以按照如下方式实现：

```python
# Python code
result = 0
for i in range(100):
    result += i
```

**注意**这里最大的不同之处：在 C 语言中，每个变量的数据类型**被明确地声明**；而在Python中，类型是**动态推断**的。这意味着可以将任何类型的数据指定给任何变量：

```python
# Python code
x = 4
x = "four"
```

这里已经将变量x的类型由整型转变成了字符串，而同样的操作在C语言中将会导致编译错误或其他未知的后果：

```C
/* C code */
int x = 4;
x = "four";  // 编译失败
```

为了学习用Python有效且高效地分析数据，下面我们将理解动态类型语言是**如何工作的**。

## Python整型不仅仅是一个整型

标准的Python实现是用C语言编写的。
每一个Python对象都是一个**聪明的伪C语言结构体**，该结构体不仅包含其值，还有其他信息。
举例，当我们在Python中定义一个整型，例如x=10000时,x并不是一个“原生”整型，而是一个**指针**，指向一个C语言的复合结构体，结构体里包含了一些值。
查看 Python 3.4 的源代码，可以发现整型（长整型）的定义，如下所示：

```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```
Python3.4中的一个整型实际上包括4个部分:

- ``ob_refcnt``,是一个引用计数，它帮助Python默默地处理内存的分配和回收。
- ``ob_type``, 将变量的类型编码
- ``ob_size``,指定接下来的数据成员的大小
- ``ob_digit``,包含我们希望 Python变量表示的实际整型值

这意味着与C语言这样的编译语言中的整型相比，在Python中存储一个整型会有一些开销，正如下图所示。

![Integer Memory Layout](figures/cint_vs_pyint.png)

这里``PyObject_HEAD``是结构体中包含引用计数、类型编码和其他之前提到的内容的部分。

由于Python的整型结构体里面还包含了大量额外的信息，所以Python可以自由、动态地编码。但是，Python类型中的这些额外信息也会成为负担，在多个对象组合的结构体中尤其明显。

## Python列表不仅仅是一个列表

Python中的标准可变多元素容器是列表。如下是一个整型值列表：

In [1]:
L = list(range(10))
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [2]:
type(L[0])

int

如下是一个字符串列表：

In [3]:
L2 = [str(c) for c in L]
L2

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [4]:
type(L2[0])

str

基于Python 的动态类型特性，甚至可以创建一个异构的列表：

In [5]:
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

[bool, str, float, int]

Python的灵活性需要付出一定代价：为了获得这些灵活的类型，列表中的每一项必须包含各自的类型信息、引用计数和其他信息；也就是说，每一项都是一个完整的Python对象。

来看一个特殊的例子，如果列表中的所有变量都是同一类型的，那么很多信息都会显得多余——将数据存储在固定类型的数组中应该会更高效。

动态类型的列表和固定类型的（NumPy式）数组间的区别如下图所示：

![Array Memory Layout](figures/array_vs_list.png)

**为什么使用Numpy?** 列表的优势是灵活，因为每个列表元素是一个包含数据和类型信息的完整结构体，而且列表可以用任意类型的数据填充。

固定类型的NumPy式数组缺乏这种灵活性，但是能更有效地存储和操作数据。

## Python中的固定类型数组

Python提供了将数据存储在有效的、固定类型的数据缓存中的选项。

例如，array数组模块：

In [6]:
import array
L = list(range(10))
A = array.array('i', L)
A

array('i', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

这里的``'i'``是一个数据类型码，表示数据为整型。


In [7]:
import numpy as np

## 从Python列表创建数组

首先，可以用 np.array 从 Python 列表创建数组：

In [8]:
import numpy as np

In [9]:
# 整型数组:
np.array([1, 4, 2, 5, 3])

array([1, 4, 2, 5, 3])

**注意**，不同于Python的列表（list），NumPy要求数组必须包含**同一类型**的数据。如果类型不匹配，NumPy将会向上转换（如果可行）。

In [10]:
np.array([3.14, 4, 2, 3])

array([3.14, 4.  , 2.  , 3.  ])

这里整型被转换为浮点型。

如果希望明确设置数组的数据类型，可以用``dtype``关键字：

In [25]:
np.array([1, 2, 3, 4], dtype='float32')

array([1., 2., 3., 4.], dtype=float32)

NumPy数组可以被指定为多维的。以下是用列表初始化多维数组的一种方法：

In [12]:
#  嵌套列表构成的多维数组
np.array([range(i, i + 3) for i in [2, 4, 6]])

array([[2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

内层的列表被当作二维数组的行。

## 从头创建数组

对于**大型数组**，用NumPy内置的方法**从头**创建数组是一种更高效的方法。

举例：

In [13]:
# 创建一个长度为10的数组，数组的值都是0 ，整型
np.zeros(10, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [14]:
# 创建一个3×5的浮点型数组，数组的值都是1 
np.ones((3, 5), dtype=float)
#注：(3, 5)是一个元组，一般用于传递一组参数。与列表不同，元组的值不能修改（防篡改）。

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [15]:
# 创建一个3×5的浮点型数组，数组的值都是3.14 
np.full((3, 5), 3.14)

array([[3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14]])

In [16]:
#  创建一个3×5的浮点型数组，数组的值是一个线性序列 
# 从0开始，到20结束，步长为2 
#  （它和内置的range()函数类似）
np.arange(0, 20, 2)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [17]:
# 创建一个5个元素的数组，这5个数均匀地分配到0~1 
np.linspace(0, 1, 5)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [18]:
# 创建一个3×3的、在0~1均匀分布的随机数组成的数组 
np.random.random((3, 3))
#第一个random表示random子模块，第二个random表示均匀分布的随机数生成函数

array([[0.74414629, 0.46638636, 0.35503381],
       [0.38496295, 0.47088079, 0.53324649],
       [0.58777046, 0.97812669, 0.35254286]])

In [19]:
#  创建一个3×3的、均值为0、方差为1的
#正态分布的随机数数组 
np.random.normal(0, 1, (3, 3))

array([[ 0.8473687 ,  0.44156308, -0.4936881 ],
       [ 1.51762424,  1.32724138, -1.1124154 ],
       [ 1.15925749, -0.6347313 ,  0.28566586]])

In [20]:
#  创建一个3×3的、[0, 10)区间的随机整型数组
np.random.randint(0, 10, (3, 3))

array([[6, 5, 8],
       [4, 5, 9],
       [6, 0, 8]])

In [21]:
# 创建一个3×3的单位矩阵 
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [28]:
#  创建一个由3个整型数组成的未初始化的数组 
# 数组的值是内存空间中的任意值 
np.empty(3)

array([1., 1., 1.])

## NumPy标准数据类型

NumPy 数组包含**同一类型**的值，因此详细了解这些数据**类型**及其**限制**是非常重要的。

下表列出了标准NumPy数据类型。注意，当构建一个数组时，你可以用一个**字符串参数**来指定数据类型：

```python
np.zeros(10, dtype='int16')
```

也可以用相关的**NumPy对象**来指定：:

```python
np.zeros(10, dtype=np.int16)
```

| Data type	    | Description |
|---------------|-------------|
| ``bool_``     | Boolean (True or False) stored as a byte |
| ``int_``      | Default integer type (same as C ``long``; normally either ``int64`` or ``int32``)| 
| ``intc``      | Identical to C ``int`` (normally ``int32`` or ``int64``)| 
| ``intp``      | Integer used for indexing (same as C ``ssize_t``; normally either ``int32`` or ``int64``)| 
| ``int8``      | Byte (-128 to 127)| 
| ``int16``     | Integer (-32768 to 32767)|
| ``int32``     | Integer (-2147483648 to 2147483647)|
| ``int64``     | Integer (-9223372036854775808 to 9223372036854775807)| 
| ``uint8``     | Unsigned integer (0 to 255)| 
| ``uint16``    | Unsigned integer (0 to 65535)| 
| ``uint32``    | Unsigned integer (0 to 4294967295)| 
| ``uint64``    | Unsigned integer (0 to 18446744073709551615)| 
| ``float_``    | Shorthand for ``float64``.| 
| ``float16``   | Half precision float: sign bit, 5 bits exponent, 10 bits mantissa| 
| ``float32``   | Single precision float: sign bit, 8 bits exponent, 23 bits mantissa| 
| ``float64``   | Double precision float: sign bit, 11 bits exponent, 52 bits mantissa| 
| ``complex_``  | Shorthand for ``complex128``.| 
| ``complex64`` | Complex number, represented by two 32-bit floats| 
| ``complex128``| Complex number, represented by two 64-bit floats| 

<!--NAVIGATION-->
< [Introduction to NumPy](02.00-Introduction-to-NumPy.ipynb) | [Contents](Index.ipynb) | [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb) >